## Semantic Search Engine

In [10]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, util
import torch
from PyPDF2 import PdfReader
import os

In [14]:
def extract_text_from_pdf(url: str) -> str:
    # creating a pdf reader object
    reader = PdfReader(url)
    
    # printing number of pages in pdf file
    text = ''    
    for page_num in range(len(reader.pages)):
        # Extract text from the current page
        page = reader.pages[page_num]
        text += page.extract_text()
    return text


In [16]:
base_path = '../data/raw/dataset/'
content = []
names = []
with os.scandir(base_path) as entries:
    for entry in entries:
        if entry.name.endswith(".pdf"):
            names.append(entry.name)
            content.append(extract_text_from_pdf(base_path+entry.name))
dataset = {'document':names, 'summary': content}